## run suite2p

Run suite2p on recording, using green channel for registration shifts, and applying them to the red channel:

In [ ]:
import os, requests
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

import suite2p
from suite2p import default_db, default_settings


In [ ]:

db = {"data_path": ["/media/carsen/disk2/suite2p_paper/GT1"], 
      "input_format": "tif",
      "nplanes": 1, # each tiff has these many planes in sequence
      "nchannels": 2, # each tiff has these many channels per plane
      "keep_movie_raw": True, # keep raw un-registered movie
}

# get info about mesoscope ROI locations in tiffs
import json
filename = Path(db["data_path"][0]).joinpath("ops.json")
with open(filename, "r") as f:
    ops_in = json.load(f)
db["lines"] = ops_in["lines"]
db["dy"] = ops_in["dy"]
db["dx"] = ops_in["dx"]

settings = default_settings()
settings["fs"] = ops_in["fs"] # sampling rate (not used in registration)
settings["torch_device"] = "cuda:0" # registration device
settings["registration"]["align_by_chan2"] = False # compute registration shifts using green channel
settings["diameter"] = ops_in["diameter"]
settings["registration"]["maxregshiftNR"] = 10 # increased from default for large nonrigid movements
settings["registration"]["batch_size"] = 250 # increased for medium/large GPUs
# don't run rest of pipeline
settings["run"]["do_detection"] = True # False
settings["run"]["do_registration"] = 2

from suite2p.run_s2p import logger_setup
logger_setup() # set up logger for logging messages

output_settings = suite2p.run_s2p(settings=settings, db=db)

### convert to tiffs for caiman and fiola

In [ ]:
## make output tiffs
from tqdm import trange
from suite2p import io 
from tifffile import imwrite
db_paths = [Path(db["data_path"][0]) / "suite2p" / f"plane{i}" / "db.npy" for i in range(3)]
print(db_paths)
for i, db_path in enumerate(db_paths):        
    db = np.load(db_path, allow_pickle=True).item()
    tiff_path = Path(db["data_path"][0]).joinpath(f"tiffs_roi{i}")
    (tiff_path / "chan1").mkdir(exist_ok=True, parents=True)
    (tiff_path / "chan2").mkdir(exist_ok=True, parents=True)
    for c in range(2):
        cstr = "" if c == 0 else "_chan2"
        raw_file = db[f"raw_file{cstr}"]
        Ly, Lx = db["Ly"], db["Lx"]
        n_frames = db["nframes"]
        with io.BinaryFile(Ly=Ly, Lx=Lx, filename=raw_file, n_frames=n_frames, write=False) as f:
            batch_size = 500
            for i in trange(0, int(np.ceil(n_frames / batch_size))):
                frames = f[i * batch_size : min(n_frames, (i+1) * batch_size)]
                imwrite(tiff_path / f"chan{c+1}" / f"raw_{i:04d}.tif", frames, imagej=True)

### get example frames and shifts for registration figure

In [ ]:
import reg_suite2p 
import importlib
importlib.reload(reg_suite2p)

root = "/media/carsen/disk2/suite2p_paper/GT1"
out = reg_suite2p.example_reg(root)
frand, freg, refImg, cc_ex, yoff, xoff, yblock, xblock, nblocks, cc_nr_ex, cc_up_ex, yxup, u, v, tPC, regPC, regDX = out
os.makedirs("results/", exist_ok=True)
dat = {"frand": frand, "freg": freg, "refImg": refImg, "cc_ex": cc_ex,
        "yoff": yoff, "xoff": xoff, "yblock": yblock,
        "xblock": xblock, "nblocks": nblocks, "cc_nr_ex": cc_nr_ex,
        "cc_up_ex": cc_up_ex, "yxup": yxup, "u": u, "v": v,
        "tPC": tPC, "regPC": regPC, "regDX": regDX,}
np.save("results/ex_reg.npy", dat)

In [ ]:
from suite2p.io import BinaryFile
from suite2p.registration.register import (compute_reference, compute_filters_and_norm, 
                                           compute_shifts, shift_frames)
import torch

root = "/media/carsen/disk2/suite2p_paper/GT1"

frames_mean, cc, imax, refImg0, refImg = reg_suite2p.example_refinit(root)

np.save('results/initial_ref.npy', {'frames_mean': frames_mean, 'cc': cc.numpy(), 'imax': imax,
                                    'refImg0': refImg0, 'refImg': refImg})

### make registration figure + suppfig for refimg

In [ ]:
import figures
dat = np.load('results/ex_reg.npy', allow_pickle=True).item()
fig = figures.registration_fig(**dat)
os.makedirs("figures", exist_ok=True)
fig.savefig("figures/fig2.pdf", dpi=150)    


dat = np.load('results/initial_ref.npy', allow_pickle=True).item()
fig = figures.suppfig_initial_ref(**dat)
fig.savefig('figures/suppfig_initial_ref.pdf', dpi=150)

## run other algs

### run caiman

Create an env with caiman:

```
conda create --name caiman caiman -c conda-forge -y
conda activate caiman
pip install natsort
```

Run the following in the env, changing path to tiffs (in example above it is `"/media/carsen/disk2/suite2p_paper/GT1"`)

```
python reg_caiman.py --root /path/to/tiffs/ --roi 0
python reg_caiman.py --root /path/to/tiffs/ --roi 1
python reg_caiman.py --root /path/to/tiffs/ --roi 2
```

### run fiola

Create an env with fiola (modified instructions from [here](https://github.com/nel-lab/FIOLA?tab=readme-ov-file#installation-guide)):

```
cd ~/
git clone https://github.com/nel-lab/FIOLA.git
git clone https://github.com/flatironinstitute/CaImAn.git -b v1.9.13
cd FIOLA
conda create --name fiola python==3.8
conda activate fiola
conda install -c nvidia cudnn=8.0 python=3.8.0
pip install -r requirements.txt 
pip install -e .
cd ../CaImAn
pip install -e . 
```

File modification also required of FIOLA repo file `gpu_mc_nnls.py`, see details [here](https://github.com/nel-lab/FIOLA?tab=readme-ov-file#demo).

(I also had the issue that tensorflow was looking for a cu10 file so I created a symbolic link from 11 to 10, with `~/lib` added to my library path: `ln -s /usr/local/cuda-11.8/lib64/libcusolver.so.11 ~/lib/libcusolver.so.10`)

Run the following in the env, changing path to tiffs

```
python reg_fiola.py --root /path/to/tiffs/ --roi 0
python reg_fiola.py --root /path/to/tiffs/ --roi 1
python reg_fiola.py --root /path/to/tiffs/ --roi 2
```

## compute registration metrics

In [ ]:
from suite2p.registration import metrics
from suite2p.io import BinaryFile
import torch 

#root = "/media/carsen/disk2/suite2p_paper/GT1/"
root = "/home/carsen/dm11_string/suite2p_paper/reg_benchmarks/GT1/"
device = torch.device("cuda")
db = np.load(os.path.join(root, 'suite2p/plane0/db.npy'), allow_pickle=True).item()
settings = np.load(os.path.join(root, 'suite2p/plane0/settings.npy'), allow_pickle=True).item()
settings = settings['registration']
Ly, Lx = db["Ly"], db["Lx"]
n_frames = db["nframes"]
print(Ly, Lx)
reg_file = db["reg_file"]   
reg_file_chan2 = db["reg_file_chan2"]   
with BinaryFile(Ly, Lx, filename=reg_file, write=False) as f:
    fr0 = f[:5000:100].copy()
with BinaryFile(Ly, Lx, filename=reg_file_chan2, write=False) as f:
    fr_20 = f[:5000:100].copy()

yrange = [20, Ly - 20]
xrange = [20, Lx - 20]
nsamp = 2000
inds = np.linspace(0, n_frames - 100, nsamp).astype("int")
regDXs = np.nan * np.zeros((3, 2, 3, 30, 4), "float32")
regPCs = np.nan * np.zeros((3, 2, 3, 2, 30, yrange[1] - yrange[0], xrange[1] - xrange[0]), "float32")
tPCs =  np.nan * np.zeros((3, 2, 3, nsamp, 30), "float32")

# dat = np.load('results/benchmarks_reg.npy', allow_pickle=True).item()
# regDXs = dat["regDXs"]
# regPCs = dat["regPCs"]
# tPCs = dat["tPCs"]

for ialg, alg in enumerate(["suite2p", "caiman", "fiola"]):
    # if alg != "suite2p":
    #     continue
    for chan in range(2):
        for iroi in range(3):
            print(alg, chan, iroi)
            chanstr = "" if chan==0 else "_chan2"
            if alg=="raw" or alg=="suite2p":
                algstr = "_raw" if alg=="raw" else ""
                bin_name = os.path.join(root, f"suite2p/plane{iroi}/data{chanstr}{algstr}.bin")
                f_reg = BinaryFile(Ly, Lx, filename=bin_name)
                mov = f_reg[inds][:, yrange[0] : yrange[-1], xrange[0] : xrange[-1]]
            elif "caiman" in alg or alg=="fiola":
                if chan==0:
                    out_mmap = os.path.join(root, f"tiffs_roi{iroi}/chan1/{alg}_reg.mmap")
                    f_reg = np.memmap(out_mmap, mode="r", dtype="float32", shape=(n_frames, Ly, Lx))
                    mov = f_reg[inds][:, yrange[0] : yrange[-1], xrange[0] : xrange[-1]]
                elif "caiman" in alg:
                    out_mmap = os.path.join(root, f"tiffs_roi{iroi}/chan2/{alg}_d1_1112_d2_650_d3_1_order_C_frames_32187.mmap")
                    f_reg = np.memmap(out_mmap, mode="r", shape=(Ly * Lx, n_frames), dtype="float32", 
                                        order="C").reshape(Lx, Ly, n_frames).transpose(2, 1, 0)
                    mov = f_reg[inds][:, yrange[0] : yrange[-1], xrange[0] : xrange[-1]]                
                else:
                    continue
            
            pclow, pchigh, sv, tPC = metrics.pclowhigh(
                mov, nlowhigh=np.minimum(300, mov.shape[0] // 2), nPC=30,
                random_state=None)
            tPCs[ialg, chan, iroi] = tPC
            pclow = torch.from_numpy(pclow).to(device).float()
            pchigh = torch.from_numpy(pchigh).to(device).float()
            regPCs[ialg, chan, iroi] = torch.stack((pclow, pchigh), dim=0).cpu().numpy()
            regDXs[ialg, chan, iroi] = metrics.pc_register(
                pclow, pchigh, smooth_sigma=settings["smooth_sigma"], block_size=settings["block_size"],
                maxregshift=settings["maxregshift"], maxregshiftNR=settings["maxregshiftNR"], 
                snr_thresh=settings["snr_thresh"], spatial_taper=settings["spatial_taper"])
            print(regDXs[ialg, chan, iroi, 0])
            

## timings

See `reg_timing_jobs.py` for all the calls for a server. Calls for suite2p:

```
for i in 500 1000 2000 4000 8000 16000 32000; do
    python reg_suite2p.py --root /path/to/tiffs/ --tfr $i
    python reg_suite2p.py --root /path/to/tiffs/ --tfr $i --rigid
done 
```

For caiman:
```
for i in 500 1000 2000 4000 8000 16000 32000; do
    python reg_caiman.py --root /path/to/tiffs/ --timing --tfr $i
    python reg_caiman.py --root /path/to/tiffs/ --timing --tfr $i --rigid
done 
```

For fiola:
```
for i in 500 1000 2000 4000 8000 16000 32000; do
    python reg_fiola.py --root /path/to/tiffs/ --timing --tfr $i
done 
```


In [ ]:
root = "/home/carsen/dm11_string/suite2p_paper/reg_benchmarks/GT1/"
alg_names = ["suite2p", "caiman_16", "fiola"]
tframes = 500 * 2**np.arange(0,7)
timings = np.nan*np.zeros((3, 2, len(tframes)))
for ialg, alg in enumerate(alg_names):
    nr = True if alg!="fiola" else False 
    for i in range(1 + nr):
        for t, tfr in enumerate(tframes): 
            try:
                timings[ialg,i,t] = np.load(os.path.join(root, f"timings/{alg}_{['rigid_',''][i]}{tfr}.npy"))
            except: 
                pass


In [ ]:
# dat = np.load("results/benchmarks_reg.npy", allow_pickle=True).item()
# regDXs = dat['regDXs']
# regPCs = dat['regPCs']
# tPCs = dat['tPCs']
# timings = dat['timings']
# alg_names = dat['alg_names'] 
# fr0 = dat['fr0']
# fr_20 = dat['fr_20']

np.save("results/benchmarks_reg.npy", {"regDXs": regDXs, "regPCs": regPCs, 
                               "tPCs": tPCs, "timings": timings,
                               "alg_names": alg_names, 
                               "fr0": fr0, "fr_20": fr_20})

In [ ]:
dat = np.load('results/benchmarks_reg.npy', allow_pickle=True).item()

In [ ]:
import figures
import importlib 
importlib.reload(figures)
fig = figures.regmetrics_fig(**dat)
fig.savefig('figures/fig3.pdf', dpi=150)

In [ ]:
# root = Path('/media/carsen/disk2/suite2p_paper/GT1/')
from fig_utils import *
root = Path("/home/carsen/dm11_string/suite2p_paper/reg_benchmarks/GT1/")

fig = plt.figure(figsize=(14,8))

for i in range(3):
    for j in range(3):
        ax = plt.subplot(3, 3, i+1 + 3*j)
        if j==0:
            reg_outputs = np.load(root / 'suite2p' / f'plane{i}' / 'reg_outputs.npy', allow_pickle=True).item()
            plt.title(f'ROI {i}', y=1.2)
            xoff = reg_outputs['xoff'] + reg_outputs['xoff1'].mean(axis=1)
            yoff = reg_outputs['yoff'] + reg_outputs['yoff1'].mean(axis=1)
        elif j==1:
            dat = np.load(root / f'tiffs_roi{i}/chan1/shifts_caiman.npy', allow_pickle=True).item()
            yoff = dat['shifts_rig'][:,0] + dat['y_shifts_els'].mean(axis=-1)
            xoff = dat['shifts_rig'][:,1] + dat['x_shifts_els'].mean(axis=-1)
        elif j==2:
            shifts = np.load(root / f'tiffs_roi{i}/chan1/shifts_fiola.npy')
            yoff, xoff = shifts.T
        xcol = np.clip(np.array(matplotlib.colors.to_rgb(alg_cols[j]))+0.2, 0, 1)
        ycol = np.clip(np.array(matplotlib.colors.to_rgb(alg_cols[j]))-0.2, 0, 1)
        ax.plot(xoff, color=xcol)
        ax.plot(yoff, color=ycol)
        ax.text(0.1, 1.05, alg_names[j], transform=ax.transAxes, ha='center', fontsize='large')
        ax.text(0.2, 0.9, 'y-offset', color=ycol, transform=ax.transAxes, ha='center')
        ax.text(0.2, 0.8, 'x-offset', color=xcol, transform=ax.transAxes, ha='center')
        ax.set_ylim([-11, 18])
        ax.set_xlim([0, len(yoff)])
        if i==0:
            ax.set_ylabel('registration offset ($\\mu$m)')
        if j==2:
            ax.set_xlabel('frame number')
plt.tight_layout()

fig.savefig('figures/suppfig_offsets.pdf', dpi=150)